In [74]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import DataCollatorWithPadding
import numpy as np


from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier


from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer


In [75]:
class ConfiguredMetric:
    def __init__(self, metric, *metric_args, **metric_kwargs):
        self.metric = metric
        self.metric_args = metric_args
        self.metric_kwargs = metric_kwargs
    
    def add(self, *args, **kwargs):
        return self.metric.add(*args, **kwargs)
    
    def add_batch(self, *args, **kwargs):
        return self.metric.add_batch(*args, **kwargs)

    def compute(self, *args, **kwargs):
        return self.metric.compute(*args, *self.metric_args, **kwargs, **self.metric_kwargs)

    @property
    def name(self):
        return self.metric.name

    def _feature_names(self):
        return self.metric._feature_names()

In [76]:

df=pd.read_csv("dataset2.csv")
df.rename(columns = {'UserID':'label',}, inplace = True)

df.head()

""""
import pandas as pd
import mysql.connector

# Database connection parameters
host = "your_host"
port = 3306  # default MySQL port
user = "your_username"
password = "your_password"
database = "your_database_name"

# Establishing the connection
conn = mysql.connector.connect(host=host, port=port, user=user, password=password, database=database)

# SQL query
query = "SELECT column1, column2, column3 FROM your_table_name"

# Fetching data into DataFrame
df = pd.read_sql(query, conn)

# Renaming columns if necessary
df.rename(columns={'Participants': 'label'}, inplace=True)

# Display the first few rows of the DataFrame
print(df.head())

# Close the connection
conn.close()
"""

'"\nimport pandas as pd\nimport mysql.connector\n\n# Database connection parameters\nhost = "your_host"\nport = 3306  # default MySQL port\nuser = "your_username"\npassword = "your_password"\ndatabase = "your_database_name"\n\n# Establishing the connection\nconn = mysql.connector.connect(host=host, port=port, user=user, password=password, database=database)\n\n# SQL query\nquery = "SELECT column1, column2, column3 FROM your_table_name"\n\n# Fetching data into DataFrame\ndf = pd.read_sql(query, conn)\n\n# Renaming columns if necessary\ndf.rename(columns={\'Participants\': \'label\'}, inplace=True)\n\n# Display the first few rows of the DataFrame\nprint(df.head())\n\n# Close the connection\nconn.close()\n'

In [77]:
df.head()

,label,NewUser,Username,Email,Gender,Age,Height,Weight,ActivityLevel,GoalID,DietPreferenceID,RegionID,AllergenID
0,28001,True,Robert Clark,robert.clark@example.com,Other,57,192,87,Sedentary,5,3,5,10
1,28002,False,Stephanie White,stephanie.white@example.com,Female,62,183,100,Very Active,2,6,6,10
2,28003,True,Amanda Davis,amanda.davis@hotmail.com,Other,45,189,61,Active,3,8,3,1
3,28004,False,Catherine Mitchell,catherine.mitchell@gmail.com,Other,65,178,56,Very Active,4,4,10,6
4,28005,True,Donna Hill,donna.hill@hotmail.com,Other,46,185,111,Moderately Active,10,7,6,9


In [78]:
def combine_columns_to_text(df, exclude_columns=None, column_weights=None):
    if exclude_columns is None:
        exclude_columns = []
    if column_weights is None:
        column_weights = {}
    if df.empty:
        return pd.DataFrame(columns=['text', 'label'])
    
    # Store the 'label' column if it exists and is not in the exclude list
    label_col = df['label'] if 'label' in df.columns and 'label' not in exclude_columns else None
    
    # Drop the columns that need to be excluded
    df_combined = df.drop(columns=exclude_columns, errors='ignore')
    
    # Apply weighting to specified columns by repeating their contents along with the column name
    for column, weight in column_weights.items():
        if column in df_combined.columns:
            df_combined[column] = df_combined[column].apply(
                lambda x: (' '.join([f"{column}:{x}"] * weight)) if pd.notnull(x) else ''
            )
    
    # Combine all columns into a single column
    df_combined['text'] = df_combined.apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)
    
    # Include the 'label' column in the result if it was stored
    if label_col is not None:
        df_combined['label'] = label_col
    
    # Return the DataFrame with the 'text' and 'label' columns
    return df_combined[['text', 'label']] if label_col is not None else df_combined[['text']]


In [79]:
def combine_columns_to_text_simple(df, exclude_columns=None):
    if exclude_columns is None:
        exclude_columns = []
    if df.empty:
        return pd.DataFrame(columns=['text', 'label'])
    
    # Store the 'label' column if it exists and is not in the exclude list
    label_col = df['label'] if 'label' in df.columns and 'label' not in exclude_columns else None
    
    # Drop the columns that need to be excluded
    df_combined = df.drop(columns=exclude_columns, errors='ignore')
    
    # Combine column names and values into a single column by concatenating the text from each column with its column name
    df_combined['text'] = df_combined.apply(lambda row: ' '.join([f"{col}:{val}" for col, val in row.dropna().items()]), axis=1)
    
    # Include the 'label' column in the result if it was stored
    if label_col is not None:
        df_combined['label'] = label_col
    
    # Return the DataFrame with the 'text' and 'label' columns
    return df_combined[['text', 'label']] if label_col is not None else df_combined[['text']]


In [123]:
# Define weights for each column
column_weights = {
    'Username': 0,
    'Email': 0,
    'Gender': 0,
    'Age': 0,
    'Heig   ht': 0,
    'Weight': 0,
    'ActivityLevel': 15,
    'GoalID': 10,
    'DietPreferenceID': 15,
    'RegionID': 15,
    'AllergenID': 30  # Higher weight for AllergenID
}

# Combine columns with specified weights
generalDataframe = combine_columns_to_text(df, exclude_columns=['DietID'], column_weights=column_weights)
generalDataframe2=combine_columns_to_text_simple(df,exclude_columns=['DietID','Email','Username'])


In [124]:
generalDataframe.head()


,text,label
0,28001 True 192 ActivityLevel:Sedentary Ac...,28001
1,28002 False 183 ActivityLevel:Very Active...,28002
2,28003 True 189 ActivityLevel:Active Activ...,28003
3,28004 False 178 ActivityLevel:Very Active...,28004
4,28005 True 185 ActivityLevel:Moderately A...,28005


In [125]:
generalDataframe2.head()

,text,label
0,label:28001 NewUser:True Gender:Other Age:57 H...,28001
1,label:28002 NewUser:False Gender:Female Age:62...,28002
2,label:28003 NewUser:True Gender:Other Age:45 H...,28003
3,label:28004 NewUser:False Gender:Other Age:65 ...,28004
4,label:28005 NewUser:True Gender:Other Age:46 H...,28005


In [83]:

# Encode labels
label_encoder = LabelEncoder()
generalDataframe['label'] = label_encoder.fit_transform(generalDataframe['label'])

# Split dataset
train_df, val_df = train_test_split(generalDataframe, test_size=0.001, random_state=42)


In [84]:
# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Tokenization
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["text"])

tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_val_dataset = val_dataset.map(preprocess_function, batched=True)

In [85]:
numberOfClasses=max(train_df['label'])

In [86]:
myDataset=Dataset.from_pandas(train_df)

In [87]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=numberOfClasses+1)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'pre_classifier.

In [88]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [89]:

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Training
trainer.train()

# Evaluation
eval_results = trainer.evaluate(tokenized_val_dataset)
print(eval_results)


c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 189/189 [54:33<00:00, 17.32s/it]   


{'train_runtime': 3272.9512, 'train_samples_per_second': 0.916, 'train_steps_per_second': 0.058, 'train_loss': 6.91446681754299, 'epoch': 3.0}


100%|██████████| 1/1 [00:00<00:00, 10.50it/s]

{'eval_loss': 6.752269744873047, 'eval_runtime': 0.2808, 'eval_samples_per_second': 3.561, 'eval_steps_per_second': 3.561, 'epoch': 3.0}


In [90]:
import torch
def predict_label(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512, padding=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    with torch.no_grad():
        logits = model(**inputs).logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    predicted_label = torch.argmax(probabilities, dim=-1).item()
    return label_encoder.inverse_transform([predicted_label])[0]

In [91]:
predicted_labelDISTILBERT=""

In [92]:
# Example user information with column names
text_to_predict = """
    Username:JohnDoe Email:johndoe@example.com Gender:Male Age:90 Height:180 Weight:80 
    ActivityLevel:Sedentary GoalID:10 DietPreferenceID:2 RegionID:8 AllergenID:10
    """
predicted_labelDISTILBERT
# Predict the label for the input text
predicted_labelDISTILBERT = predict_label(text_to_predict)

# Print the predicted label
print(f"The predicted label for the input text is: {predicted_labelDISTILBERT}")


The predicted label for the input text is: 28927


In [93]:
X_train, X_test, y_train, y_test = train_test_split(generalDataframe['text'], generalDataframe['label'], random_state=123)


In [94]:
vectorizer = TfidfVectorizer(stop_words='english', strip_accents='unicode', lowercase=True)
X_train_transformed = vectorizer.fit_transform(X_train)
X_test_transformed = vectorizer.transform(X_test)


In [95]:
modelNb=MultinomialNB()
modelNb.fit(X_train_transformed,X_train)
predictions=modelNb.predict(X_test_transformed)

In [96]:
svm_model_linear = SVC(kernel = 'linear', C = 1).fit(X_train_transformed, X_train)
svm_predictions = svm_model_linear.predict(X_test_transformed)

In [97]:
knn = KNeighborsClassifier(n_neighbors = 12).fit(X_train_transformed, X_train)
knn_predictions = knn.predict(X_test_transformed) 

In [98]:

def predict_label_id_multinomial_nb(text):
    text_transformed = vectorizer.transform([text])
    return modelNb.predict(text_transformed)[0]

def predict_label_id_svm(text):
    text_transformed = vectorizer.transform([text])
    return svm_model_linear.predict(text_transformed)[0]

def predict_label_id_knn(text):
    text_transformed = vectorizer.transform([text])
    return knn.predict(text_transformed)[0]


In [99]:
text_to_predict = """
    Username:JohnDoe Email:johndoe@example.com Gender:Male Age:90 Height:180 Weight:80 
    ActivityLevel:Sedentary GoalID:10 DietPreferenceID:2 RegionID:10 AllergenID:10
    """

In [100]:

# Predict using Logistic Regression
predicted_labelMNB=""


In [101]:

# Predict using Multinomial Naive Bayes
predicted_labelMNB= predict_label_id_multinomial_nb(text_to_predict)
print(f"Multinomial Naive Bayes predicted label ID: {predicted_labelMNB}")


Multinomial Naive Bayes predicted label ID: 28023 True     150  ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary GoalID:10 GoalID:10 GoalID:10 GoalID:10 GoalID:10 GoalID:10 GoalID:10 GoalID:10 GoalID:10 GoalID:10 DietPreferenceID:2 DietPreferenceID:2 DietPreferenceID:2 DietPreferenceID:2 DietPreferenceID:2 DietPreferenceID:2 DietPreferenceID:2 DietPreferenceID:2 DietPreferenceID:2 DietPreferenceID:2 DietPreferenceID:2 DietPreferenceID:2 DietPreferenceID:2 DietPreferenceID:2 DietPreferenceID:2 RegionID:6 RegionID:6 RegionID:6 RegionID:6 RegionID:6 RegionID:6 RegionID:6 RegionID:6 RegionID:6 RegionID:6 RegionID:6 RegionID:6 RegionID:6 RegionID:6 RegionID:6 AllergenID:10 AllergenID:1

In [102]:

predicted_labelSVM=""
# Predict using SVM


In [103]:
predicted_labelSVM = predict_label_id_svm(text_to_predict)
print(f"SVM predicted label ID: {predicted_labelSVM}")


SVM predicted label ID: 28017 False     150  ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary GoalID:10 GoalID:10 GoalID:10 GoalID:10 GoalID:10 GoalID:10 GoalID:10 GoalID:10 GoalID:10 GoalID:10 DietPreferenceID:10 DietPreferenceID:10 DietPreferenceID:10 DietPreferenceID:10 DietPreferenceID:10 DietPreferenceID:10 DietPreferenceID:10 DietPreferenceID:10 DietPreferenceID:10 DietPreferenceID:10 DietPreferenceID:10 DietPreferenceID:10 DietPreferenceID:10 DietPreferenceID:10 DietPreferenceID:10 RegionID:6 RegionID:6 RegionID:6 RegionID:6 RegionID:6 RegionID:6 RegionID:6 RegionID:6 RegionID:6 RegionID:6 RegionID:6 RegionID:6 RegionID:6 RegionID:6 RegionID:6 AllergenID:10 AllergenID:10 Al

In [104]:
# Predict using KNN
predicted_labelKNN = predict_label_id_knn(text_to_predict)
print(f"KNN predicted label ID: {predicted_labelKNN}")

KNN predicted label ID: 28001 True     192  ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary GoalID:5 GoalID:5 GoalID:5 GoalID:5 GoalID:5 GoalID:5 GoalID:5 GoalID:5 GoalID:5 GoalID:5 DietPreferenceID:3 DietPreferenceID:3 DietPreferenceID:3 DietPreferenceID:3 DietPreferenceID:3 DietPreferenceID:3 DietPreferenceID:3 DietPreferenceID:3 DietPreferenceID:3 DietPreferenceID:3 DietPreferenceID:3 DietPreferenceID:3 DietPreferenceID:3 DietPreferenceID:3 DietPreferenceID:3 RegionID:5 RegionID:5 RegionID:5 RegionID:5 RegionID:5 RegionID:5 RegionID:5 RegionID:5 RegionID:5 RegionID:5 RegionID:5 RegionID:5 RegionID:5 RegionID:5 RegionID:5 AllergenID:10 AllergenID:10 AllergenID:10 AllergenID:10 

In [133]:
print(predicted_labelDISTILBERT)
print(predicted_labelSVM)
print(predicted_labelKNN)
print(predicted_labelMNB)


28927
28017 False     150  ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary ActivityLevel:Sedentary GoalID:10 GoalID:10 GoalID:10 GoalID:10 GoalID:10 GoalID:10 GoalID:10 GoalID:10 GoalID:10 GoalID:10 DietPreferenceID:10 DietPreferenceID:10 DietPreferenceID:10 DietPreferenceID:10 DietPreferenceID:10 DietPreferenceID:10 DietPreferenceID:10 DietPreferenceID:10 DietPreferenceID:10 DietPreferenceID:10 DietPreferenceID:10 DietPreferenceID:10 DietPreferenceID:10 DietPreferenceID:10 DietPreferenceID:10 RegionID:6 RegionID:6 RegionID:6 RegionID:6 RegionID:6 RegionID:6 RegionID:6 RegionID:6 RegionID:6 RegionID:6 RegionID:6 RegionID:6 RegionID:6 RegionID:6 RegionID:6 AllergenID:10 AllergenID:10 AllergenID:10 Allerg

In [134]:
import re

# Example predicted label strings for SVM, KNN, and MNB models
predictedLabelSVM = predicted_labelSVM
predictedLabelKNN = predicted_labelKNN
predictedLabelMNB = predicted_labelMNB

# Function to extract the leading integer from a predicted label string
def extract_leading_integer_from_label(predicted_label):
    match = re.match(r'(\d+)', predicted_label)
    if match:
        return int(match.group(1))  # Convert the matched digits to an integer
    else:
        return None  # Return None if no leading integer is found

# Extract leading integers for each predicted label
leading_integer_svm = extract_leading_integer_from_label(predictedLabelSVM)
leading_integer_knn = extract_leading_integer_from_label(predictedLabelKNN)
leading_integer_mnb = extract_leading_integer_from_label(predictedLabelMNB)

# Print the extracted integers
print("Leading integer for SVM label:", leading_integer_svm)
print("Leading integer for KNN label:", leading_integer_knn)
print("Leading integer for MNB label:", leading_integer_mnb)




Leading integer for SVM label: 28017
Leading integer for KNN label: 28001
Leading integer for MNB label: 28023


In [143]:
# Assuming the leading integers for SVM, KNN, and MNB models are stored in variables:
# leading_integer_svm, leading_integer_knn, leading_integer_mnb
# And the predicted label for DistilBERT is stored in:
# predictedLabelDistilBERT

# Create a dictionary with model names as keys and their respective values
model_values_dict = {
    "DistilBERT": predicted_labelDISTILBERT,  # This is the predicted label for DistilBERT
    "SVM": leading_integer_svm,  # This is the leading integer extracted from the SVM predicted label
    "KNN": leading_integer_knn,  # This is the leading integer extracted from the KNN predicted label
    "MNB": leading_integer_mnb   # This is the leading integer extracted from the MNB predicted label
}

# Print the dictionary to verify its contents
for model, value in model_values_dict.items():
    print(f"{model}: {value}")


DistilBERT: 28927
SVM: 28017
KNN: 28001
MNB: 28023


In [136]:
# Define variables to hold the label values you want to filter by
filter_label_value_svm = leading_integer_svm
filter_label_value_distilbert = predicted_labelDISTILBERT
filter_label_value_mnb = leading_integer_mnb
filter_label_value_knn = leading_integer_knn

# Filter the DataFrame for each predicted label and retrieve the corresponding 'text' value

# For predicted_labelSVM
filtered_df_svm = generalDataframe2.loc[generalDataframe2['label'] == filter_label_value_svm, 'text']
corresponding_text_value_svm = filtered_df_svm.iloc[0] if not filtered_df_svm.empty else "No matching text found for the specified label"

# For predicted_labelDISTILBERT
filtered_df_distilbert = generalDataframe2.loc[generalDataframe2['label'] == filter_label_value_distilbert, 'text']
corresponding_text_value_distilbert = filtered_df_distilbert.iloc[0] if not filtered_df_distilbert.empty else "No matching text found for the specified label"

# For predicted_labelMNB
filtered_df_mnb = generalDataframe2.loc[generalDataframe2['label'] == filter_label_value_mnb, 'text']
corresponding_text_value_mnb = filtered_df_mnb.iloc[0] if not filtered_df_mnb.empty else "No matching text found for the specified label"

# For predicted_labelKNN
filtered_df_knn = generalDataframe2.loc[generalDataframe2['label'] == filter_label_value_knn, 'text']
corresponding_text_value_knn = filtered_df_knn.iloc[0] if not filtered_df_knn.empty else "No matching text found for the specified label"

# Print or use the corresponding text values as needed
print("SVM:", corresponding_text_value_svm)
print("DistilBERT:", corresponding_text_value_distilbert)
print("MNB:", corresponding_text_value_mnb)
print("KNN:", corresponding_text_value_knn)


SVM: label:28017 NewUser:False Gender:Other Age:56 Height:150 Weight:80 ActivityLevel:Sedentary GoalID:10 DietPreferenceID:10 RegionID:6 AllergenID:10
DistilBERT: label:28927 NewUser:True Gender:Male Age:53 Height:161 Weight:141 ActivityLevel:Sedentary GoalID:9 DietPreferenceID:2 RegionID:10 AllergenID:7
MNB: label:28023 NewUser:True Gender:Male Age:64 Height:150 Weight:126 ActivityLevel:Sedentary GoalID:10 DietPreferenceID:2 RegionID:6 AllergenID:10
KNN: label:28001 NewUser:True Gender:Other Age:57 Height:192 Weight:87 ActivityLevel:Sedentary GoalID:5 DietPreferenceID:3 RegionID:5 AllergenID:10


In [149]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

# Initialize the model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Your info string
info_string = text_to_predict  # Make sure text_to_predict is defined with your actual text

# Dictionary mapping model names to their corresponding text values
model_texts = {
    "DistilBERT": corresponding_text_value_distilbert,
    "SVM": corresponding_text_value_svm,
    "MNB": corresponding_text_value_mnb,
    "KNN": corresponding_text_value_knn
}

# Filter out empty predicted texts
model_texts = {model_name: text for model_name, text in model_texts.items() if text.strip()}

# Proceed only if there are non-empty predicted labels
if model_texts:
    # Encode the info string to get its embedding
    info_embedding = model.encode(info_string, convert_to_tensor=True)
    
    # Initialize a dictionary to store similarity scores for each model
    similarity_scores = {}

    # Variable to store the text of the most similar model
    most_similar_text = None
    
    # Iterate over each model text, compute its similarity to the info_string, and store it
    for model_name, text in model_texts.items():
        # Encode the model's text to get its embedding
        text_embedding = model.encode(text, convert_to_tensor=True)
        
        # Compute cosine similarity
        similarity_score = util.pytorch_cos_sim(info_embedding, text_embedding).item()
        
        # Store the similarity score
        similarity_scores[model_name] = similarity_score
        
        # Print the model name and similarity score with the info_string
        print(f"Model: {model_name}, Similarity Score: {similarity_score}")

        # Update the most similar text if this score is the highest
        if most_similar_text is None or similarity_score > similarity_scores.get(most_similar_model, 0):
            most_similar_model = model_name
            most_similar_text = text
            highest_similarity_score = similarity_score

    # Print the most similar model, its text, and similarity score
    print(f"\nThe most similar model to the info string is: {most_similar_model} with text: '{most_similar_text}' and a similarity score of: {highest_similarity_score}")
else:
    print("All predicted labels are empty.")


Model: DistilBERT, Similarity Score: 0.6777118444442749
Model: SVM, Similarity Score: 0.6325041055679321
Model: MNB, Similarity Score: 0.6718852519989014
Model: KNN, Similarity Score: 0.6807601451873779

The most similar model to the info string is: KNN with text: 'label:28001 NewUser:True Gender:Other Age:57 Height:192 Weight:87 ActivityLevel:Sedentary GoalID:5 DietPreferenceID:3 RegionID:5 AllergenID:10' and a similarity score of: 0.6807601451873779


In [150]:
print(text_to_predict)
print(most_similar_text)


    Username:JohnDoe Email:johndoe@example.com Gender:Male Age:90 Height:180 Weight:80 
    ActivityLevel:Sedentary GoalID:10 DietPreferenceID:2 RegionID:10 AllergenID:10
    
label:28001 NewUser:True Gender:Other Age:57 Height:192 Weight:87 ActivityLevel:Sedentary GoalID:5 DietPreferenceID:3 RegionID:5 AllergenID:10
